In [2]:
import numpy as np
from matplotlib import pyplot as plt

In [55]:
dir = "/home/michel/Documents/Minimal/build/collisions/cart"

# t x y z vx vy vz
data = np.loadtxt(f'{dir}/{4}.dat')
x = data[:,2]
y = data[:,3]
z = data[:,4]

In [56]:
fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')
ax.plot(x, y, z)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()

In [63]:
dir = '/home/michel/Documents/Minimal/build/collisions'

%matplotlib qt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for i in range(1, 50):
	data = np.loadtxt(f'{dir}/{i}.dat')
	r = data[:,1]
	q = data[:,2]
	z = data[:,3]

	# print(i, np.max(z))

	if i != 30:
		ax.plot(r * np.cos(q), r * np.sin(q) , z)		

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()	

This one form above doesn't make sense for the tf = 1200, dt=0.00001, nskip=200, obs_skip=20000 because in only 1.2 the ion slows down to ~zero, and that is, in this case  data points

In [15]:
dir = "/home/michel/Documents/Minimal/build/collisions/cart"

%matplotlib qt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x = np.array([])
y = np.array([])
z = np.array([])


for i in range(1, 100):	
	data = np.loadtxt(f'{dir}/{i}.dat')
	
	x = np.append(x, data[-1, 4])
	y = np.append(y, data[-1, 5])
	z = np.append(z, data[-1, 6])
	# z = data[:,3]

ax.scatter(x, y, z)		

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.set_aspect('auto', adjustable='box')

plt.show()	

Now a simulation with just dispersion

In [62]:
%matplotlib qt

dir = '/home/michel/Documents/Minimal/build/dispersion'


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

vr = np.array([])
vq = np.array([])
vz = np.array([])
q = np.array([])


for i in range(1, 99):
	if i == 30: continue
	
	data = np.loadtxt(f'{dir}/{i}.dat')
	vr = np.append(vr, data[-1, 4])
	vq = np.append(vq, data[-1, 5])
	vz = np.append(vz, data[-1, 6])
	q = np.append(q, data[-1, 2])
	# z = data[:,3]

vx = vr * np.cos(q) - vq * np.sin(q)
vy = vr * np.sin(q) + vq * np.cos(q)

# ax.scatter(vx, vy)		
# ax.scatter(vx, vz)
# ax.scatter(vy, vz)

v2 = np.square(vx) + np.square(vy) + np.square(vz)
print(f'<v^2> = {np.mean(v2)} standar deviation {np.std(v2)}')

print(f'means:\nvx = {np.mean(vx)}\nvy = {np.mean(vy)}\nvz = {np.mean(vz)}')

ax.scatter(vx, vy, vz)		

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# ax.set_aspect('equal', adjustable='box')

plt.show()

<v^2> = 0.0015425906584521148 standar deviation 0.0013796561761603135
means:
vx = -0.0003899982846019612
vy = -0.0021588598880502765
vz = -0.0011946550927835053


In [58]:
# prepare to continue simulations

dir = '/home/michel/Documents/Minimal/build/dispersion_only'

with open(f'{dir}/initial_conditions.dat', 'w') as file:
	for i in range(1, 100):
		data = np.loadtxt(f'{dir}/{i}.dat')[-1]
		line = ""
		for x in data:
			line += str(x) + ' '
		file.write(line + '\n')


### Dispersion difusion coefficient

In [66]:
dir = '/home/michel/Documents/Minimal/build/dispersion_only'

data = np.loadtxt(f'{dir}/1_car.dat')
vx = data[:,4]
vy = data[:,5]
vz = data[:,6]

In [67]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.plot(vx, vy, vx)

plt.show()

In [27]:
vs = np.transpose(np.array([vx, vy, vz]))

In [37]:
def mean_sqr_diff(vs):
	ans = []
	st_dev = []

	for n in range(len(vs)):
		sqr_diff = []

		for i in range(len(vs) - n):
			diff = vs[i + n] - vs[i]
			sqr_diff.append(np.dot(diff, diff))
		
		ans.append(np.mean(sqr_diff))
		st_dev.append(np.std(sqr_diff))
	
	return np.array(ans), np.array(st_dev)


In [47]:
msd, msd_std = mean_sqr_diff(vs)

In [71]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.sans-serif": ["Computer Modern Sans Serif"]})

In [81]:
fig = plt.figure()
ax = fig.add_subplot(111)

dt = 0.0186
t = np.arange(0, len(msd)) * dt

ax.plot(t, msd)
ax.grid()

ax.set_xlabel('$\Delta t\ [s]$', fontsize=16)
ax.set_ylabel('$\langle \Delta \\vec{v} \Delta \\vec{v} \\rangle \ [v_0^2]$', fontsize=16)

plt.show()

In [92]:
vmods = [np.dot(x, x) for x in vs]

In [94]:
np.mean(vmods)

5.458416255674037

### Perpendicular and parallel separatelly

In order to separate parallel and perpendicular the average of vector in the range of the difference will be taken as the effective velocity direction 

In [152]:
def mean_sqr_diff_perp_and_par(vs):
	sqr_diff_par = [[] for i in range(len(vs))]
	sqr_diff_perp = [[] for i in range(len(vs))]

	for start in range(len(vs)):

		v_sum = vs[start]

		for stop in range(start, len(vs)):

			v_sum += vs[stop]

			e_mean = v_sum / np.sqrt(np.dot(v_sum, v_sum))

			diff = vs[stop] - vs[start]
			diff_par = np.dot(diff, e_mean) # scalar
			diff_perp = np.cross(diff, e_mean) # vector

			sqr_diff_par[stop - start].append(diff_par ** 2)
			sqr_diff_perp[stop - start].append(np.dot(diff_perp, diff_perp))

	diff_par = np.array(sqr_diff_par, dtype='object')
	diff_perp = np.array(sqr_diff_perp, dtype='object')

	mean = np.vectorize(np.mean)
	std = np.vectorize(np.std)

	return mean(diff_par), mean(diff_perp), std(diff_par), std(diff_perp)

# Dont use this ^^^^

too slow, made a compiled C++ version

In [160]:
msd_par, msd_perp, std_par, st_perp = mean_sqr_diff_perp_and_par(vs)

In [161]:
np.savetxt('msd_par.dat', msd_par)
np.savetxt('msd_perp.dat', msd_perp)
np.savetxt('std_par.dat', std_par)
np.savetxt('std_perp.dat', st_perp)

In [163]:
fig = plt.figure()
ax = fig.add_subplot(111)

dt = 0.0186
t = np.arange(0, len(msd_perp)) * dt

ax.plot(t, msd_perp, label='perpendicular')
# ax.plot(t, msd_par, label='parallel')
ax.grid()

ax.set_xlabel('$\Delta t\ [s]$', fontsize=16)
ax.set_ylabel('$\langle \Delta \\vec{v} \Delta \\vec{v} \\rangle \ [v_0^2]$', fontsize=16)

plt.legend()

plt.show()

In [154]:
msd_par

array([0.00000000e+00, 2.65961402e+19, 4.29275380e+19, 5.38841574e+19,
       6.37433483e+19, 7.71094556e+19, 9.68005459e+19, 1.29304835e+20,
       1.92006354e+20, 3.34586326e+20])